# 프로그래머스 강의
github - [corazzon-github](https://github.com/corazzon/KaggleStruggle/)
## NLP with TF-IDF
* tf-idf를 통해 벡터화 하고 xbgoost를 사용해본다

## 전처리
* 병렬처리 코드만 사용
* HTML tag만 전처리해준다

In [1]:

import pandas as pd
"""
header = 0 은 파일의 첫 번째 줄에 열 이름이 있음을 나타내며 
delimiter = \t 는 필드가 탭으로 구분되는 것을 의미한다.
quoting = 3은 쌍따옴표를 무시하도록 한다.
"""
# QUOTE_MINIMAL (0), QUOTE_ALL (1), 
# QUOTE_NONNUMERIC (2) or QUOTE_NONE (3).

# 레이블인 sentiment 가 있는 학습 데이터
train = pd.read_csv('../input/kaggle_data/Bag_of_Words_Meets_Bags_Of_Popcorn/labeledTrainData.tsv', 
                    header=0, delimiter='\t', quoting=3)
# 레이블이 없는 테스트 데이터
test = pd.read_csv('../input/kaggle_data/Bag_of_Words_Meets_Bags_Of_Popcorn/testData.tsv', 
                   header=0, delimiter='\t', quoting=3)
print(train.shape)
print(test.shape)

(25000, 3)
(25000, 2)


In [2]:

# 긍정과 부정 리뷰가 어떻게 들어있는지 카운트 해본다.
# 긍정과 부정리뷰가 각각 동일하게 12,500개씩 들어있다.
train['sentiment'].value_counts()

1    12500
0    12500
Name: sentiment, dtype: int64

In [3]:
train.columns

Index(['id', 'sentiment', 'review'], dtype='object')

In [4]:
train['sentiment'].value_counts() # 긍정/부정 갯수가 반반

1    12500
0    12500
Name: sentiment, dtype: int64

In [5]:
from KaggleWord2VecUtility import KaggleWord2VecUtility
from bs4 import BeautifulSoup
from nltk.stem import WordNetLemmatizer

wordnet_lemmatizer = WordNetLemmatizer()

def review_to_words( raw_review ):
    review_text = BeautifulSoup(raw_review, 'html.parser').get_text()
    review_text = wordnet_lemmatizer.lemmatize(review_text)
    return review_text

In [11]:
train['review_clean'] = train['review'].apply(lambda x: review_to_words(x))
test['review_clean'] = test['review'].apply(lambda x: review_to_words(x))

In [13]:
X_train = train['review_clean']
X_test = test['review_clean']

## TF-IDF

TF(단어 빈도, term frequency)는 특정한 단어가 문서 내에 얼마나 자주 등장하는지를 나타내는 값으로, 이 값이 높을수록 문서에서 중요하다고 생각할 수 있다. 하지만 단어 자체가 문서군 내에서 자주 사용되는 경우, 이것은 그 단어가 흔하게 등장한다는 것을 의미한다. 이것을 DF(문서 빈도, document frequency)라고 하며, 이 값의 역수를 IDF(역문서 빈도, inverse document frequency)라고 한다. TF-IDF는 TF와 IDF를 곱한 값이다.

IDF 값은 문서군의 성격에 따라 결정된다. 예를 들어 '원자'라는 낱말은 일반적인 문서들 사이에서는 잘 나오지 않기 때문에 IDF 값이 높아지고 문서의 핵심어가 될 수 있지만, 원자에 대한 문서를 모아놓은 문서군의 경우 이 낱말은 상투어가 되어 각 문서들을 세분화하여 구분할 수 있는 다른 낱말들이 높은 가중치를 얻게 된다.

역문서 빈도(IDF)는 한 단어가 문서 집합 전체에서 얼마나 공통적으로 나타나는지를 나타내는 값이다. 전체 문서의 수를 해당 단어를 포함한 문서의 수로 나눈 뒤 로그를 취하여 얻을 수 있다.

In [14]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from nltk.corpus import words

In [17]:
import nltk
nltk.download('words')

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\hyooo\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\words.zip.


True

In [18]:
vectorizer = CountVectorizer(analyzer = 'word', 
                             lowercase = True,
                             tokenizer = None,
                             preprocessor = None,
                             stop_words = 'english',
                             min_df = 2, # 토큰이 나타날 최소 문서 개수로 오타나 자주 나오지 않는 특수한 전문용어 제거에 좋다. 
                             ngram_range=(1, 3),
                             vocabulary = set(words.words()), # nltk의 words를 사용하거나 문서 자체의 사전을 만들거나 선택한다. 
                             max_features = 90000
                            )
vectorizer

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=90000, min_df=2,
        ngram_range=(1, 3), preprocessor=None, stop_words='english',
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None,
        vocabulary={'shankings', 'quacky', 'appropre', 'prebestow', 'fibrousness', 'inopine', 'overbillow', 'rhizocaul', 'colinear', 'polyacid', 'condescendence', 'laniary', 'trachychromatic', 'unlunar', 'wingstem', 'aspermia', 'cuff', 'balatong', 'snarl', 'squirelike', 'neckmold', 'gentianella', 'hootay', ...asia', 'Pliohippus', 'dyschroia', 'wonderment', 'debauchee', 'supplely', 'parbuckle', 'bustle', 'S'})

## TF-IDF Transformer

* norm='l2' 각 문서의 피처 벡터를 어떻게 벡터 정규화 할지 정한다.
    * L2 벡터의 각 원소의 제곱의 합이 1이 되도록 만드는 것이 기본 값
    * L1 벡터의 각 원소의 절댓값의 합이 1이 되도록 크기를 조절
    
* smooth_idf = False
    * 피처를 만들 때 0으로 나오는 항목에 대해 작으 값을 더해서(스무딩을 해서) 피처를 만들지 아니면 그냥 생성할지를 결정

* sublinear_tf = False
* use_idf = True
    * TF-IDF 를 사용해 피처를 만든 것인지 아니면 단어 빈도 자체를 사용할 것인지 여부
    ```
    pipeline = Pipeline([
        ('vect', vectorizer),
        ('tfidf', TfidfTransformer(smooth_idf=False)),
    ])
    ```
    

In [19]:
pipeline = Pipeline([
        ('vect', vectorizer),
        ('tfidf', TfidfTransformer(smooth_idf=False)),
    ])

In [20]:
pipeline

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=90000, min_df=2,
        ngram_range=(1, 3), preprocessor=None, stop_words='english',
       ...('tfidf', TfidfTransformer(norm='l2', smooth_idf=False, sublinear_tf=False,
         use_idf=True))])

In [21]:
%time X_train_tfidf_vector = pipeline.fit_transform(X_train)

C:\Users\hyooo\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1067: RuntimeWarning: divide by zero encountered in true_divide
  idf = np.log(float(n_samples) / df) + 1.0


Wall time: 13.3 s


In [24]:
X_train_tfidf_vector

<25000x235892 sparse matrix of type '<class 'numpy.float64'>'
	with 1612861 stored elements in Compressed Sparse Row format>

In [25]:
vocab = vectorizer.get_feature_names()
print(len(vocab))
vocab[:10]

235892


['A',
 'Aani',
 'Aaron',
 'Aaronic',
 'Aaronical',
 'Aaronite',
 'Aaronitic',
 'Aaru',
 'Ab',
 'Ababdeh']

In [26]:
%time X_test_tfidf_vector = pipeline.fit_transform(X_test)

Wall time: 17.3 s


C:\Users\hyooo\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1067: RuntimeWarning: divide by zero encountered in true_divide
  idf = np.log(float(n_samples) / df) + 1.0


In [27]:
import numpy as np
dist = np.sum(X_train_tfidf_vector, axis=0)

for tag, count in zip(vocab, dist):
    print(count, tag)
    
pd.DataFrame(dist, columns=vocab)

[[0. 0. 0. ... 0. 0. 0.]] A


,A,Aani,Aaron,Aaronic,Aaronical,Aaronite,Aaronitic,Aaru,Ab,Ababdeh,...,zymotechnical,zymotechnics,zymotechny,zymotic,zymotically,zymotize,zymotoxic,zymurgy,zythem,zythum
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(
    n_estimators = 100, n_jobs=-1, random_state=2018)
forest

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=2018, verbose=0,
            warm_start=False)

In [29]:
%time forest = forest.fit(X_train_tfidf_vector, train['sentiment'])

Wall time: 3min 55s


## Cross Validation 교차 검증

In [30]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
k_fold = KFold(n_splits=5, shuffle=True, random_state=2018)

In [35]:
%time score = np.mean(cross_val_score(\
    forest, X_train_tfidf_vector, \
    train['sentiment'], cv=k_fold, scoring='roc_auc', n_jobs=-1))

KeyboardInterrupt: 

In [ ]:
'{:,.5f}'.format(score)

In [31]:
%time result = forest.predict(X_test_tfidf_vector)

Wall time: 2.52 s


In [33]:
output = pd.DataFrame(data={'id':test['id'], 'sentiment':result})
output.head()

output_sentiment = output['sentiment'].value_counts()
print(output_sentiment[0] - output_sentiment[1])
output_sentiment

-376


1    12688
0    12312
Name: sentiment, dtype: int64

In [34]:
output.to_csv('../output/tutorial_4_tfidf_{0:.5f}.csv'.format(score), index=False, quoting=3)

NameError: name 'score' is not defined

## XGBoost

* 2015년 캐글 블로그에 xgboost를 사용하여 17건의 우승 솔루션이 공유됨 : [Dato Winners’ Interview: 1st place, Mad Professors | No Free Hunch](http://blog.kaggle.com/2015/12/03/dato-winners-interview-1st-place-mad-professors/)
* 2016년 논문이 등록 됨 : [XGBoostArxiv.pdf](https://www.coursehero.com/file/27539150/XGBoostArxivpdf/)
* 공식문서 : XGBoost Documents
* **분산형 그래디언트 부스팅 알고리즘**
* 부스팅 알고리즘은?
    * 부스팅 알고리즘은 **약한 예측모형들을 결합**하여 강한 예측모형을 만드는 알고리즘
    * 배깅과 유사하게 초기 샘플데이터로 다수의 분류기를 만들지만 배깅과 다르게 <u>순차적</u>이다.
    * 결정트리(Decision Tree) 알고리즘의 연장선에 있음
    * 여러 개의 결정트리를 묶어 강력한 모델을 만드는 앙상블 방법
    * 분류와 회귀에 사용할 수 있음
    * 무작위성이 없으며 강력한 사전 가지치기를 사용
    * 참고 이미지 : http://www.birc.co.kr/2017/02/06/%EC%95%99%EC%83%81%EB%B8%94ensemble-%EB%B6%80%EC%8A%A4%ED%8C%85boosting/
    * 배깅과 부스팅의 차이점은 udacity에서 설명한 영상이 가장 도움이 되었음
        * 배깅 : https://www.youtube.com/watch?v=2Mg8QD0F1dQ
        * 부스팅 : https://www.youtube.com/watch?v=GM3CDQfQ4sw
* 타이타닉 경진대회에 사용 예제가 있음 XGBoost example (Python) | Kaggle

### XGBoost 참고
* A Gentle Introduction to XGBoost for Applied Machine Learning - Machine Learning Mastery

* https://speakerdeck.com/datasciencela/tianqi-chen-xgboost-overview-and-latest-news-la-meetup-talk

* datacamp의 XGboost 온라인 강의 : Extreme Gradient Boosting with XGBoost

In [36]:
import xgboost as xgb

dtrain = xgb.DMatrix(X_train_tfidf_vector, label=train['sentiment'])

In [37]:
# 멀티프로세싱은 nthread 였는데 n_jobs로 변경되었다고 한다.
# 설치 된 xgboost버전에 따라 파라메터가 다를 수 있으니 
# 이 코드를 돌리고 터미널에서 $ top -o cpu 로 CPU자원을 100%넘게 사용하고 있는지 확인해 본다.
params = {
    'booster': 'gblinear',
    'objective': 'multi:softmax',
    'eval_metric': 'merror',
    'eta' : 0.02,
    'lambda': 2.0,
    'alpha': 1.0,
    'lambda_bias': 6.0,
    'num_class': 5,
    'n_jobs' : 4,
    'silent': 1,
}

%time booster = xgb.train(params, dtrain, num_boost_round=100) # round를 지정

Wall time: 6.76 s


In [38]:
dtest = xgb.DMatrix(X_test_tfidf_vector)

result = booster.predict(dtest)

print(result.shape)
result[0:10]

(25000,)


array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [39]:
result = result.astype(int)

In [40]:
output = pd.DataFrame(data={'id':test['id'], 'sentiment':result})
output.head()

,id,sentiment
0,"""12311_10""",0
1,"""8348_2""",0
2,"""5828_4""",0
3,"""7186_2""",0
4,"""12128_7""",0


In [48]:
output['sentiment'].value_counts()
# 모두 0밖에 없음.. 이상 -> round를 300으로 바꾸면 다르게 나올것

0    25000
Name: sentiment, dtype: int64

#### Kaggle API 사용하기

In [ ]:
# !kaggle competitions submit -c word2vec-nlp-tutorial -f data/tutorial_4_..csv -m 'num_boost_round=300'

In [ ]:
# !kaggle competitions submissions -c word2vec-nlp-tutorial